In [3]:
#Import libraries
import pandas as pd
import numpy as np
import json
import time

In [4]:
#Function to convert epoch time
def convert_millis_epoch_to_datetime(epoch):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch // 1000))

In [5]:
#Load and analysis of Brands.json
#Since cpg had 2 variables, it has been seperated into 2 columns
brands = []
with open('K:/Mana/Assignments/brands.json/brands.json') as f:
    for line in f:
        brand = json.loads(line)
        brand["_id"] = brand["_id"]["$oid"]
        if("cpg" in brand):
            brand["cpg_id"] = brand["cpg"]["$id"]["$oid"]
            brand["cpg_ref"] = brand["cpg"]["$ref"]
            del brand["cpg"]
        brands.append(brand)
        
brands = pd.DataFrame(brands)
brands.head()

,_id,barcode,category,categoryCode,name,topBrand,cpg_id,cpg_ref,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac114be37ce2ead437550,Cogs,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,False,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [6]:
#Brands has 1167 rows and 8 columns
brands.shape

(1167, 9)

In [7]:
#Number of null values - primary key id has no null
brands.isnull().sum(axis = 0)

_id               0
barcode           0
category        155
categoryCode    650
name              0
topBrand        612
cpg_id            0
cpg_ref           0
brandCode       234
dtype: int64

In [8]:
#Count of unique values on BrandCode
bcCount = len(brands['brandCode'].unique())
print(bcCount)

898


In [9]:
#Exploration of category and categoryCode values
categoryCount = brands['category'].unique()
print(categoryCount)

ccCount = brands['categoryCode'].unique()
print(ccCount)

#Observation: Almost similar values in both columns

['Baking' 'Beverages' 'Candy & Sweets' 'Condiments & Sauces'
 'Canned Goods & Soups' nan 'Magazines' 'Breakfast & Cereal'
 'Beer Wine Spirits' 'Health & Wellness' 'Beauty' 'Baby' 'Frozen'
 'Grocery' 'Snacks' 'Household' 'Personal Care' 'Dairy'
 'Cleaning & Home Improvement' 'Deli' 'Beauty & Personal Care'
 'Bread & Bakery' 'Outdoor' 'Dairy & Refrigerated']
['BAKING' 'BEVERAGES' 'CANDY_AND_SWEETS' nan 'HEALTHY_AND_WELLNESS'
 'GROCERY' 'PERSONAL_CARE' 'CLEANING_AND_HOME_IMPROVEMENT'
 'BEER_WINE_SPIRITS' 'BABY' 'BREAD_AND_BAKERY' 'OUTDOOR'
 'DAIRY_AND_REFRIGERATED' 'MAGAZINES' 'FROZEN']


In [10]:
#Exploration of cpg_id and cpg_ref values
cpg_idCount = len(brands['cpg_id'].unique())
print(cpg_idCount)

cpg_refCount = brands['cpg_ref'].unique()
print(cpg_refCount)

#Observation: Almost similar values in both columns

196
['Cogs' 'Cpgs']


In [11]:
#Load and analysis of Users.json
users = []
with open('K:/Mana/Assignments/users.json/users.json') as f:
    for line in f:
        user = json.loads(line)
        user["_id"] = user["_id"]["$oid"]
        user["createdDate"] = convert_millis_epoch_to_datetime(user["createdDate"]["$date"])
        if("lastLogin" in user):
            user["lastLogin"] = convert_millis_epoch_to_datetime(user["lastLogin"]["$date"])
        users.append(user)
users =  pd.DataFrame(users)
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 09:24:04,2021-01-03 09:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 09:24:04,2021-01-03 09:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 09:24:04,2021-01-03 09:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 09:25:30,2021-01-03 09:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 09:24:04,2021-01-03 09:25:37,consumer,Email,WI


In [12]:
#Users has 495 rows and 7 columns
users.shape

(495, 7)

In [13]:
#Number of null values - primary key id has no null
users.isnull().sum(axis = 0)

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [17]:
#For now, 2 kinds of roles can be observed, 
#but it would be optimized if roles were to be used as a seperate table considering more roles may be added in the future
users['role'].unique()

array(['consumer', 'fetch-staff'], dtype=object)

In [20]:
#Load and analysis of Receipts.json
#Contained a itemList, which has been stored as a seperated dataframe
receipts = []
receipt_items_list = []


with open('K:/Mana/Assignments/receipts.json/receipts.json') as f:
    for line in f:
        rc = json.loads(line)
        rc["_id"] = rc["_id"]["$oid"]
        rc["createDate"] = convert_millis_epoch_to_datetime(rc["createDate"]["$date"])
        if("dateScanned" in rc):
            rc["dateScanned"] = convert_millis_epoch_to_datetime(rc["dateScanned"]["$date"])
        if("finishedDate" in rc):
            rc["finishedDate"] = convert_millis_epoch_to_datetime(rc["finishedDate"]["$date"])
        if("modifyDate" in rc):
            rc["modifyDate"] = convert_millis_epoch_to_datetime(rc["modifyDate"]["$date"])
        if("pointsAwardedDate" in rc):
            rc["pointsAwardedDate"] = convert_millis_epoch_to_datetime(rc["pointsAwardedDate"]["$date"])
        if("purchaseDate" in rc):
            rc["purchaseDate"] = convert_millis_epoch_to_datetime(rc["purchaseDate"]["$date"])
        
        if("rewardsReceiptItemList" in rc):
            for each_item in rc["rewardsReceiptItemList"]:        
                each_item["receipt_id"] = rc["_id"]
                receipt_items_list.append(each_item)
        
        receipts.append(rc)
        
    
receipts = pd.DataFrame(receipts)
receipts.head()



,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 09:25:31,2021-01-03 09:25:31,2021-01-03 09:25:31,2021-01-03 09:25:36,2021-01-03 09:25:31,500.0,2021-01-02 18:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 09:24:43,2021-01-03 09:24:43,2021-01-03 09:24:43,2021-01-03 09:24:48,2021-01-03 09:24:43,150.0,2021-01-02 09:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 09:25:37,2021-01-03 09:25:37,NaN,2021-01-03 09:25:42,NaN,5,2021-01-02 18:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 09:25:34,2021-01-03 09:25:34,2021-01-03 09:25:34,2021-01-03 09:25:39,2021-01-03 09:25:34,5.0,2021-01-02 18:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 09:25:06,2021-01-03 09:25:06,2021-01-03 09:25:11,2021-01-03 09:25:11,2021-01-03 09:25:06,5.0,2021-01-02 09:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


In [21]:
receipt_items = pd.DataFrame(receipt_items_list)
receipt_items.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#Receipts had 1119 rows and 15 columns
receipts.shape

(1119, 15)

In [23]:
#Number of null values - primary key id has no null
#But a lot of other columns have null values, this is a data quality issue. 
receipts.isnull().sum(axis = 0)

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

In [24]:
receipt_items.shape

(6941, 35)

In [25]:
#More than 90% of the dataset has NaN values, which needs to be addressed since it is a data quality issue 
##and can lead to unnecessary allocation of storage space. 
receipt_items.isnull().sum(axis = 0)

barcode                               3851
description                            381
finalPrice                             174
itemPrice                              174
needsFetchReview                      6128
partnerItemId                            0
preventTargetGapPoints                6583
quantityPurchased                      174
userFlaggedBarcode                    6604
userFlaggedNewItem                    6618
userFlaggedPrice                      6642
userFlaggedQuantity                   6642
receipt_id                               0
needsFetchReviewReason                6722
pointsNotAwardedReason                6601
pointsPayerId                         5674
rewardsGroup                          5210
rewardsProductPartnerId               4672
userFlaggedDescription                6736
originalMetaBriteBarcode              6870
originalMetaBriteDescription          6931
brandCode                             4341
competitorRewardsGroup                6666
discountedI

In [26]:
##Both of these columns in receipt_items have similar values. Need more information to keep or discard one/both the columns.
#There are many other columns in the similar fashion, they needed to be evaluated 
##on the business use case it provides for optimzied DB design
print(receipt_items['rewardsProductPartnerId'].unique())
print(receipt_items['pointsPayerId'].unique())

[nan '5332f5fbe4b03c9a25efd0ba' '5332f5f3e4b03c9a25efd0ae'
 '5332f709e4b03c9a25efd0f1' '559c2234e4b06aca36af13c6'
 '5e7cf838f221c312e698a628' '5332f5f6e4b03c9a25efd0b4'
 '5e825d64f221c312e698a62a' '550b2565e4b001d5e9e4146f'
 '5a734034e4b0d58f376be874' '5d9b4f591dda2c6225a284aa'
 '5eebc5412455c97a877ef382' '5fb6b608be37ce522e165cb9'
 '53e10d6368abd3c7065097cc' '5f58fa84be37ce21f1287c0f'
 '5f036cedac99f02d72388690' '5ffdb69ebe37ce746df1192a']
[nan '5332f5fbe4b03c9a25efd0ba' '5332f5f3e4b03c9a25efd0ae'
 '5332f709e4b03c9a25efd0f1' '5332f5f6e4b03c9a25efd0b4'
 '559c2234e4b06aca36af13c6' '550b2565e4b001d5e9e4146f'
 '5a734034e4b0d58f376be874' '5eebc5412455c97a877ef382'
 '5fb6b608be37ce522e165cb9' '53e10d6368abd3c7065097cc'
 '5f58fa84be37ce21f1287c0f' '5e825d64f221c312e698a62a'
 '5f036cedac99f02d72388690' '5e7cf838f221c312e698a628'
 '5ffdb69ebe37ce746df1192a']
